In [ ]:
import numpy as np
import pandas as pd

import boto3
import sagemaker

In [ ]:
session = sagemaker.Session(default_bucket='quant-sagemaker-test-roles')
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = "eu-central-1"
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {bucket}")
print(f"sagemaker session region: {region}")

In [ ]:
train_input_path = f"s3://quant-sagemaker-test-roles/imdb/data/small/train.csv"
test_input_path = f"s3://quant-sagemaker-test-roles/imdb/data/small/test.csv"

## Training

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters = {'epochs': 1,
                   'per_device_train_batch_size': 32,
                   'model_name': 'distilbert-base-uncased'
                   }

# create the Estimator
estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./source',
    instance_type='ml.g4dn.xlarge',  # Note: needs to be an instance with gpu
    instance_count=1,
    role=role,
    transformers_version='4.5',
    tensorflow_version='2.4',
    py_version='py37',
    hyperparameters=hyperparameters,
    sagemaker_session=session,
)
estimator.fit({'train': train_input_path, 'test': test_input_path})

## Inference

In [ ]:
# model_data = estimator.model_data
model_data = 's3://quant-sagemaker-test-roles/huggingface-tensorflow-training-2021-08-06-08-52-15-250/output/model.tar.gz'
print(model_data)

In [ ]:
from sagemaker.huggingface.model import HuggingFaceModel

model = HuggingFaceModel(
    model_data=model_data,
    role=role,
    transformers_version='4.6',
    tensorflow_version='2.4',
    py_version='py37',
    source_dir="source",
    entry_point="inference.py",
    sagemaker_session=session,
)


### Endpoint

In [ ]:
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.medium",
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.StringDeserializer(),
)

In [ ]:
input_1 = "this is a very good movie"
predictor.predict(input_1)

In [ ]:
input_2 = ['this movie sucks', 'this movie is ok']
predictor.predict(input_2)

In [ ]:
input_3 = ['such a terrible movie', 'what a great movie', 'omg best movie ever']
predictor.predict(input_3)

In [ ]:
predictor.delete_endpoint()

### Batch Transform

#### High level

In [ ]:
# Just a 200-line csv file needs "ml.m5.2xlarge"? Crazy
transformer = model.transformer(instance_count=1, instance_type="ml.m5.large", accept='text/csv',
                                        strategy='SingleRecord')

In [ ]:
transformer.transform(test_input_path, content_type='text/csv', split_type='Line')

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./

In [ ]:
res = pd.read_csv("test.csv.out", header=None)
res

#### Low level

In [ ]:
sagemaker = boto3.client('sagemaker')
job_name = 'pytorch-inference-2021-06-01-19-51-46-244'
prefix = 'imdb/batch/data'

In [ ]:
% % time

from time import gmtime, strftime

batch_job_name = "Batch-Transform-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
input_location = test_input_path
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, batch_job_name)

request = {
    "TransformJobName": batch_job_name,
    "ModelName": job_name,
    "BatchStrategy": "SingleRecord",
    "TransformOutput": {
        "S3OutputPath": output_location,
        "Accept": "text/csv",
        "AssembleWith": "Line",
    },
    "TransformInput": {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": input_location}},
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None",
    },
    "TransformResources": {"InstanceType": "ml.m5.large", "InstanceCount": 1},
}

sagemaker.create_transform_job(**request)
print("Created Transform job with name: ", batch_job_name)

# Wait until the job finishes
try:
    sagemaker.get_waiter("transform_job_completed_or_stopped").wait(TransformJobName=batch_job_name)
finally:
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response["TransformJobStatus"]
    print("Transform job ended with status: " + status)
    if status == "Failed":
        message = response["FailureReason"]
        print("Transform failed with the following error: {}".format(message))
        raise Exception("Transform job failed")

In [ ]:
!aws s3 cp --recursive $output_location./

### Local

In [ ]:
% % sh -s $model_data
rm - rf model
mkdir model
aws s3 cp $1 model /
tar xvzf model/model.tar.gz - -directory ./model

In [ ]:
!pip install "sagemaker>=2.31.0" "transformers==4.4.2" "datasets[s3]==1.5.0" --upgrade

In [ ]:
# pytorch_model.bin stores fine-tuned huggingface
# tokenizer_config.json stores tokenizer used for training

from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained('model')
tokenizer = AutoTokenizer.from_pretrained('model')

In [ ]:
inputs = ['this is a terrific movie']

In [ ]:
tokenized_input = tokenizer(inputs, truncation=True, padding=True)

In [ ]:
input_ids = torch.Tensor(tokenized_input['input_ids']).long()
attention_mask = torch.Tensor(tokenized_input['attention_mask']).long()

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(input_ids, attention_mask).logits
    res = np.argmax(logits)
    print('logits:', logits, 'neg/pos:', res)